In [1]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/home/nikhil/Desktop/sem8/IP/main/data/glove/glove.6B.300d.txt')
tmp_file = get_tmpfile("word2vec_300.txt")
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [63]:
import numpy as np
def checkin(a,b):
    for i in range(len(b)):
        if b[i]==a:
            return 1
    return 0

def checkchild(b,word):
    for i in range(len(b)):
        if b[i]==word:
            return 1
    return 0

def checkra(s,shead,b,bhead):
    if (len(s))==1:
        return 0
    else:
        if bhead[0]==s[-1]:
            if checkchild(shead,b[0])==0:
                return 1
    return 0

def checkla(s,shead,b,bhead):
    if (len(s))==1:
        return 0
    #elif shead[-1]==:
    #    return 0
    else:
        if shead[-1]==b[0]:
            if checkchild(bhead,s[-1])==0:
                return 1
    return 0

In [64]:
#stack,buff,staPOS,buffPOS,staDEP,buffDEP,dictPOS,dictDEP
def feature(s,b,sPOS,bPOS,sDEP,bDEP,dictPOS,dictDEP):
    x1=np.array([[0]*300]*1, np.dtype(float))
    try:
        for item in s:
            x1=x1+model[item]
    except:
        pass
    x2=np.array([[0]*300]*1, np.dtype(float))
    try:
        for item in b:
            x2=x2+model[item]
    except:
        pass
    x3=np.array([[0]*len(dictPOS)]*1, np.dtype(float))
    for item in sPOS:
        x3[0][dictPOS[item]]=1
    
    x4=np.array([[0]*len(dictPOS)]*1, np.dtype(float))
    for item in bPOS:
        x4[0][dictPOS[item]]=1
        
    x5=np.array([[0]*len(dictDEP)]*1, np.dtype(float))
    for item in sDEP:
        x5[0][dictDEP[item]]=1
        
    x6=np.array([[0]*len(dictDEP)]*1, np.dtype(float))
    for item in bDEP:
        x6[0][dictDEP[item]]=1
        
    x=np.concatenate((x1,x2,x3,x4,x5,x6),axis=1)
    return x    
    
def op_ra(s,sh,b,bh,bp,sp,bd,sd):
    s.pop()
    sh.pop()
    sp.pop()
    sd.pop()
    return [s,sh,b,bh,bp,sp,bd,sd]
def op_la(s,sh,b,bh,bp,sp,bd,sd):
    b[0]=s.pop()
    bp[0]=sp.pop()
    bd[0]=sd.pop()
    bh[0]=sh.pop()
    return [s,sh,b,bh,bp,sp,bd,sd]
def op_shift(s,sh,b,bh,bp,sp,bd,sd):
    s.append(b[0])
    sh.append(bh[0])
    sp.append(bp[0])
    sd.append(bd[0])
    del(b[0])
    del(bh[0])
    del(bp[0])
    del(bd[0])
    return [s,sh,b,bh,bp,sp,bd,sd]

In [65]:
train=open('train.conllu','r').read();
train = train.split('\n')
dictPOS={}
dictDEP={}
j=0
for i in range(60000):
    try:
        if train[i][0] != '#':
            train[i]=train[i].split('\t')
            if(train[i][6]!='_'):
                if train[i][3] in dictPOS:
                    dictPOS[train[i][3]]=dictPOS[train[i][3]]+1
                else:
                    dictPOS[train[i][3]]=1
                if train[i][7] in dictDEP:
                    dictDEP[train[i][7]]=dictDEP[train[i][7]]+1
                else:
                    dictDEP[train[i][7]]=1
                    
    except:
        pass
i=0
for items in dictPOS:
    dictPOS[items]=i;
    i=i+1
i=0
for items in dictDEP:
    dictDEP[items]=i
    i=i+1

In [66]:
train=open('train.conllu','r').read();
train = train.split('\n')

In [67]:
t=[]
l=[]
t.append([])
l.append([])
pos=[]
dep=[]
pos.append([])
dep.append([])
j=0;
for i in range(60000):
    try:
       
        if train[i][0] != '#':
            train[i]=train[i].split('\t')
            if(train[i][6]!='_'):
                t[j].append(train[i][1])
                l[j].append(train[i][6])
                pos[j].append(train[i][3])
                dep[j].append(train[i][7])
    except:
        t.append([])
        l.append([])
        pos.append([])
        dep.append([])
        j=j+1
        pass

In [68]:
X=[]
Y=[]
for i in range(len(l)):
    stack=['#root#']
    stahead=[-1]
    buff=t[i]
    buffhead=[]
    for j in range(len(l[i])):
        if l[i][j]>0:
            #print j
            buffhead.append(t[i][int(l[i][j])-1])
        else:
            buffhead.append('#root#')
    buffPOS=pos[i]
    buffDEP=dep[i]
    staPOS=[]
    staDEP=[]
#    print i
#    print buff
#    print buffhead
#    print stack
#    print stahead
    while len(buff)>0:
        if checkra(stack,stahead,buff,buffhead)==1:
            [stack,stahead,buff,buffhead,buffPOS,staPOS,buffDEP,staDEP]=op_ra(stack,stahead,buff,buffhead,buffPOS,staPOS,buffDEP,staDEP)
            X.append(feature(stack,buff,staPOS,buffPOS,staDEP,buffDEP,dictPOS,dictDEP))
            Y.append('ra')
        elif checkla(stack,stahead,buff,buffhead)==1:
            [stack,stahead,buff,buffhead,buffPOS,staPOS,buffDEP,staDEP]=op_la(stack,stahead,buff,buffhead,buffPOS,staPOS,buffDEP,staDEP)
            X.append(feature(stack,buff,staPOS,buffPOS,staDEP,buffDEP,dictPOS,dictDEP))
            Y.append('la')
        else:
            [stack,stahead,buff,buffhead,buffPOS,staPOS,buffDEP,staDEP]=op_shift(stack,stahead,buff,buffhead,buffPOS,staPOS,buffDEP,staDEP)
            X.append(feature(stack,buff,staPOS,buffPOS,staDEP,buffDEP,dictPOS,dictDEP))
            Y.append('shift')
            

In [69]:
Yt=np.array([[0]*3]*len(Y), np.dtype(float))
for i in range(len(Y)):
    if(Y[i]=='shift'):
        Yt[i][0]=1
    elif(Y[i]=='ra'):
        Yt[i][1]=1
    else:
        Yt[i][2]=1

In [70]:
len(X)

65997

In [71]:
Xtrain=np.array([[0]*730]*50000, np.dtype(float))
for i in range(50000):
    Xtrain[i]=X[i]

In [72]:
Xtest=np.array([[0]*730]*15997, np.dtype(float))
for i in range(15997):
    Xtest[i]=X[i+50000]

In [73]:
Ytrain=np.array([[0]*3]*50000, np.dtype(float))
for i in range(50000):
    Ytrain[i]=Yt[i]
Ytest=np.array([[0]*3]*15997, np.dtype(float))
for i in range(15997):
    Ytest[i]=Yt[i+50000]

In [74]:
np.save('Xtrain',Xtrain)
np.save('Ytrain',Ytrain)
np.save('Xtest',Xtest)
np.save('Ytest',Ytest)

In [76]:
Xtrain[:,0]

array([0., 0., 0., ..., 0., 0., 0.])